In [ ]:
#pip install https://sparkflows-release.s3.amazonaws.com/fire/jupyter-docker/firenotebookwheel/fire_notebook-3.1.0-py3-none-any.whl

: 

In [ ]:
#!pip install pyspark
#import pyspark

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from fire_notebook.output.workflowcontext import RestWorkflowContext
parameters_list = sys.argv
#print(len(sys.argv), "parameters_list")
'''
if len(sys.argv) >= 3:
    #restworkflowcontext = RestWorkflowContext(parameters=parameters_list)
else:
    #restworkflowcontext = RestWorkflowContext(debug=False)
'''
restworkflowcontext = RestWorkflowContext(parameters=parameters_list)

message="20"
restworkflowcontext.outputProgress(9, title="Progress", progress=message)

# IMPORTANT CHANGE PATH WHEN PUSHING TO DOCKERS
#FILE_PATH = "/content/drive/MyDrive/data/churn.all" #google coollab
FILE_PATH = "churn.all" #Analytical Docker App

# Read the CSV file, treating empty strings as NaN and forcing 'churned' to be read as a string
colnames=["state","account_length","area_code","phone_number","intl_plan","voice_mail_plan","number_vmail_messages","total_day_minutes","total_day_calls","total_day_charge","total_eve_minutes","total_eve_calls","total_eve_charge","total_night_minutes","total_night_calls","total_night_charge","total_intl_minutes","total_intl_calls","total_intl_charge","number_customer_service_calls","churned"]
df = pd.read_csv(FILE_PATH, names=colnames, header=None, na_values=[''], keep_default_na=False, dtype={'churned': str})


# Define a function to filter the DataFrame based on user-specified criteria
def filter_dataframe(df, state_filter, intl_plan_filter, voice_mail_filter):
    # Create a copy of the DataFrame to avoid modifying the original
    filtered_df = df.copy()

    # Filter by state if a specific state is provided
    if state_filter != '%':
        filtered_df = filtered_df[filtered_df['state'] == state_filter]
        print('After intl_plan_filter',filtered_df.size)

    # Filter by international plan if a specific plan is provided
    if intl_plan_filter != '%':
        print ("intl_plan_filter is:",intl_plan_filter)
        filtered_df = filtered_df[filtered_df['intl_plan'] == intl_plan_filter]
        print('After intl_plan_filter',filtered_df.size)

    # Filter by voice mail plan if a specific plan is provided
    if voice_mail_filter != '%':
        filtered_df = filtered_df[filtered_df['voice_mail_plan'] == voice_mail_filter]
        print('After voice_mail_filter',filtered_df.size)

    # Return the filtered DataFrame
    return filtered_df

# Define the filter criteria (these would typically come from user input in a web application)
state_filter = restworkflowcontext.getParmeters(parameter_name="arg_state", default='%')
intl_plan_filter = restworkflowcontext.getParmeters(parameter_name="arg_intl_plan", default='%')
voice_mail_filter = restworkflowcontext.getParmeters(parameter_name="arg_voice_mail_plan", default='%')
#churned_filter = restworkflowcontext.getParmeters(parameter_name="arg_churned", default='%')

state_filter = 'AZ'
intl_plan_filter = ' yes'
voice_mail_filter = ' yes'
'''
churned_filter = '%'
'''

# Call the filter function with the specified criteria
filtered_df = filter_dataframe(df, state_filter, intl_plan_filter, voice_mail_filter)

# Define a list of columns for which summary statistics will be computed
summary_cols = [
    # List of columns to include in the summary statistics
    "total_day_minutes", "total_day_calls", "total_day_charge", "total_eve_calls",
    "number_vmail_messages", "total_eve_minutes", "total_eve_charge",
    "total_night_minutes", "total_night_calls", "total_night_charge",
    "total_intl_minutes", "total_intl_calls", "total_intl_charge",
    "number_customer_service_calls"
]

# Compute summary statistics for the filtered DataFrame
summary_df = filtered_df[summary_cols].agg(['count', 'mean', 'min', 'max', 'std', 'var']).round(3)

# Print the summary statistics
print("Summary Statistics for Filtered Data:\n", summary_df)

restworkflowcontext.outPandasDataframe(9, "Summary Statistics for Filtered Data", summary_df)

# Drop rows with null values from the filtered DataFrame
df_drop_null = filtered_df.dropna()

# Select numeric and boolean columns for the correlation matrix calculation
numeric_bool_cols = df_drop_null.select_dtypes(include=['number', 'bool']).columns

# Calculate the correlation matrix for the numeric and boolean columns
corr_matrix = df_drop_null[numeric_bool_cols].corr()

# Print the correlation matrix
print("\nCorrelation Matrix:\n", corr_matrix)
restworkflowcontext.outPandasDataframe(9, "Correlation Matrix Table", corr_matrix)


# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix')
plt.show()

# Group the filtered DataFrame by 'churned' and compute aggregated statistics
groupby_df = filtered_df.groupby('churned').agg({
    'total_day_charge': 'sum',
    'total_eve_charge': 'sum',
    'total_night_charge': 'sum',
    'total_intl_charge': 'sum'
}).reset_index()

# Print the grouped data
print("\nGrouped by churned:\n", groupby_df.head())
restworkflowcontext.outPandasDataframe(9, "Churned Statistics", groupby_df.head())


# Create a pie chart to visualize the distribution of churned and non-churned customers
churn_counts = filtered_df['churned'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(churn_counts.values, labels=churn_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Churn Distribution')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.show()

# Create a box plot to visualize the distribution of customer service calls by churn status
plt.figure(figsize=(10, 6))
sns.boxplot(x='churned', y='number_customer_service_calls', data=filtered_df)
plt.title('Number of Customer Service Calls by Churn Status')
plt.show()

# Create a count plot to visualize the distribution of international plan subscriptions by churn status
plt.figure(figsize=(10, 6))
sns.countplot(x='intl_plan', hue='churned', data=filtered_df)
plt.title('International Plan Subscription by Churn Status')
plt.show()

progress_percentage = "100"
restworkflowcontext.outputProgress(id=9, title="Progress", progress=progress_percentage)

message = "Success."
restworkflowcontext.outSuccess(9, title="Success", text=message)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
plotly.offline.init_notebook_mode(connected=True)


In [ ]:
fig_corr = px.imshow(
    corr_matrix,
    # Remove the old 'RdBu_r' line if it was there
    color_continuous_scale=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0',
                            '#FFB2C8', '#FFD5BE', '#C7D2FF', '#83E3F4', '#D1BFFF',
                            '#649BF7', '#F8C9EE', '#219AAD', '#FFBFA3'],
    zmin=-1, zmax=1,
    title="Correlation Matrix"
)

plot_html_corr = plotly.offline.plot(fig_corr, output_type='div', include_plotlyjs=False)
restworkflowcontext.outPlotly(9, "Correlation Matrix Plot", plot_html_corr)

# Show in Jupyter Notebook
fig_corr.show()


In [ ]:
fig_pie = px.pie(
    names=churn_counts.index,
    values=churn_counts.values,
    title='Churn Distribution',
    hole=0 , # 0 means full pie; change to 0.3 for donut chart,
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3'] 
)
plot_html_pie = plotly.offline.plot(fig_pie, output_type='div', include_plotlyjs=False)
restworkflowcontext.outPlotly(9, "Churn Distribution Plot", plot_html_pie)

fig_pie.show()


In [ ]:
fig_box = px.box(
    filtered_df,
    x='churned',
    y='number_customer_service_calls',
    title='Number of Customer Service Calls by Churn Status',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3'] 
)
plot_html_box = plotly.offline.plot(fig_box, output_type='div', include_plotlyjs=False)
restworkflowcontext.outPlotly(9, "Customer Service Calls Box Plot", plot_html_box)

fig_box.show()


In [ ]:
fig_count = px.histogram(
    filtered_df,
    x='intl_plan',
    color='churned',
    barmode='group',
    title='International Plan Subscription by Churn Status',
    color_discrete_sequence=['#77C8D0', '#FFE3B2', '#FAA19B', '#A1BDFD', '#B6D7D0','#FFB2C8','#FFD5BE','#C7D2FF','#83E3F4','#D1BFFF','#649BF7','#F8C9EE','#219AAD','#FFBFA3']
)
plot_html_count = plotly.offline.plot(fig_count, output_type='div', include_plotlyjs=False)
restworkflowcontext.outPlotly(9, "International Plan Count Plot", plot_html_count)

fig_count.show()


In [ ]:
#fig_group = px.bar(
#    groupby_df,
#    x='churned',
#    y=['total_day_charge', 'total_eve_charge', 'total_night_charge', 'total_intl_charge'],
#    title='Charges by Churn Status',
#    barmode='group'
#)
#plot_html_group = plotly.offline.plot(fig_group, output_type='div', include_plotlyjs=False)
#restworkflowcontext.outPlotly(9, "Charges by Churn Status", plot_html_group)

#fig_group.show()
